### loading the packages

In [1]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

from sklearn.cluster import KMeans

### read the data

In [2]:
df = pd.read_csv('train_E6oV3lV.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train_E6oV3lV.csv'

In [ ]:
corpus = list(df['tweet'])
corpus = corpus[0:100]

### Clustering

In [ ]:
corpus_embeddings = embedder.encode(corpus)

In [ ]:
num_clusters = 2
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_


### Result

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")